<a href="https://colab.research.google.com/github/dubuel/MLPytorch/blob/main/dubuel./gradientdescent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 경사 하강법을 이용한 얕은 신경망 학습


In [7]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [8]:
EPOCHS = 1000

## 네트워크 구조 정의
### 얕은 신경망
#### 입력 계층 : 2, 은닉 계층 : 128 (Sigmoid activation), 출력 계층 : 10 (Softmax activation)

In [9]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.d1 = tf.keras.layers.Dense(128, input_dim = 2 ,activation= 'sigmoid')
    self.d2 = tf.keras.layers.Dense(10, activation= 'softmax')

  def call( self, x, training = None, mask = None):
    x = self.d1(x)
    return self.d2(x)


## 학습 루프 정의

In [10]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer,train_loss, train_metric):
  with tf.GradientTape() as tape:
    predictions = model(inputs)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_metric(labels, predictions)

## 데이터셋 생성, 전처리

In [26]:
np.random.seed(0)

pts= list()
labels = list()
center_pts = np.random.uniform(-8.0, 8.0, (10,2))
for label, center_pts in enumerate(center_pts):
  for _ in range(100):
    pts.append(center_pts + np.random.randn(*center_pts.shape))
    labels.append(label)

pts = np.stack(pts,axis = 0).astype(np.float32)
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices((pts, labels)).shuffle(1000).batch(32)

## 모델 생성

In [41]:
model = MyModel()

## 손실 함수 및 최적화 알고리즘 설정
### CrossEntropy, Adam Optimizer

In [44]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 평가 지표 설정
### Accuracy

In [43]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

## 학습 루프

In [47]:
for epoch in range(EPOCHS):
  for x, label in train_ds:
    train_step(model,x,label,loss_object,optimizer,train_loss, train_accuracy)
  print(f'Epoch {epoch+1}, Loss: {train_loss.result()}, accuracy: {train_accuracy.result()}')

Epoch 1, Loss: 0.3916032910346985, accuracy: 0.8745967149734497
Epoch 2, Loss: 0.39095211029052734, accuracy: 0.8746561408042908
Epoch 3, Loss: 0.3903430104255676, accuracy: 0.8747408390045166
Epoch 4, Loss: 0.38969993591308594, accuracy: 0.874855637550354
Epoch 5, Loss: 0.3890479803085327, accuracy: 0.8749538064002991
Epoch 6, Loss: 0.38842231035232544, accuracy: 0.8750408291816711
Epoch 7, Loss: 0.38778749108314514, accuracy: 0.8751370906829834
Epoch 8, Loss: 0.3871854841709137, accuracy: 0.8752323985099792
Epoch 9, Loss: 0.3866157531738281, accuracy: 0.8753267526626587
Epoch 10, Loss: 0.3859946131706238, accuracy: 0.8753901124000549
Epoch 11, Loss: 0.3854043483734131, accuracy: 0.8754528760910034
Epoch 12, Loss: 0.3848082423210144, accuracy: 0.8755447268486023
Epoch 13, Loss: 0.3842226266860962, accuracy: 0.875606119632721
Epoch 14, Loss: 0.3836404085159302, accuracy: 0.8756914138793945
Epoch 15, Loss: 0.383039265871048, accuracy: 0.8757709860801697
Epoch 16, Loss: 0.382452875375747

## 데이터셋 및 학습 파라미터 저장

In [48]:
np.savez_compressed('dataset.npz',inputs = pts, labels = labels)

W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()
W_h = np.transpose(W_h) #쓸떄랑의 괴리를 없애기 위함
W_o = np.transpose(W_o)
np.savez_compressed('parameters.npz',
                    W_h = W_h,
                    b_h = b_h,
                    W_o = W_o,
                    b_o = b_o)
